# Credit Risk Resampling Techniques

In [237]:
import warnings
warnings.filterwarnings('ignore')

In [197]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# Read the CSV into DataFrame

In [198]:
# Load the data
file_path = Path('lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [199]:
#binary encoding
df = pd.get_dummies(df, columns=["homeowner"])
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk,1,0,0


# Split the Data into Training and Testing

In [200]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [201]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [202]:
# Check the balance of our target values
df['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [203]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [204]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
data_scaler = StandardScaler()

In [205]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_data_scaler = data_scaler.fit(X_train)


In [206]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_data_scaler.transform(X_train)
X_test_scaled = X_data_scaler.transform(X_test)

# Simple Logistic Regression

In [207]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)
Counter(y_train)

Counter({'low_risk': 56277, 'high_risk': 1875})

In [208]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [209]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [210]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [211]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)


# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [212]:
X_resampled.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10100.0,7.402,50300,0.403579,4,1,20300,1,0,0
1,9200.0,7.049,46900,0.360341,3,0,16900,1,0,0
2,8800.0,6.870,45200,0.336283,3,0,15200,1,0,0
3,8900.0,6.919,45700,0.343545,3,0,15700,1,0,0
4,8500.0,6.731,43900,0.316629,3,0,13900,0,1,0


In [213]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

model_Random = LogisticRegression(solver='lbfgs', random_state=1)
model_Random.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [214]:
X_resampled.shape

(112554, 10)

In [215]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_nro = model_Random.predict(X_test)
balanced_accuracy_score(y_test, y_pred_nro)

0.9947746894823818

In [216]:
# Display the confusion matrix
# YOUR CODE HERE
cm_nro = confusion_matrix(y_test, y_pred_nro)
cm_nro

array([[  622,     3],
       [  106, 18653]])

In [217]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_nro))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [218]:
# Resample the training data with SMOTE
# YOUR CODE HERE
X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train)
# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [219]:
X_resampled_smote.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10100.0,7.402,50300,0.403579,4,1,20300,1,0,0
1,9200.0,7.049,46900,0.360341,3,0,16900,1,0,0
2,8800.0,6.870,45200,0.336283,3,0,15200,1,0,0
3,8900.0,6.919,45700,0.343545,3,0,15700,1,0,0
4,8500.0,6.731,43900,0.316629,3,0,13900,0,1,0


In [220]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
smote_model = LogisticRegression(solver='lbfgs', random_state=1)
smote_model.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(random_state=1)

In [221]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_smote = smote_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smote)

0.9948279972279972

In [222]:
# Display the confusion matrix
# YOUR CODE HERE
cm_smote = confusion_matrix(y_test, y_pred_smote)
cm_smote

array([[  622,     3],
       [  104, 18655]])

In [223]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [224]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
cc = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled_cc)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [225]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=1)

In [226]:
# Calculate the balanced accuracy score
# YOUR CODE HERE

y_pred_cc = model_cc.predict(X_test)
cm_cc = confusion_matrix(y_test, y_pred_cc)


In [227]:
# Display the confusion matrix
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred_cc) 

0.9828813049736127

In [228]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.96       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [229]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
sm = SMOTEENN(random_state=1)
X_resampled_sm, y_resampled_sm = sm.fit_resample(X_train, y_train)
# View the count of target classes with Counter
# YOUR CODE HERE

Counter(y_resampled_sm)

Counter({'high_risk': 55290, 'low_risk': 55900})

In [230]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_sm = LogisticRegression(solver='lbfgs', random_state=1)
model_sm.fit(X_resampled_sm, y_resampled_sm)

LogisticRegression(random_state=1)

In [234]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred_sm = model_sm.predict(X_test)
cm_sm = confusion_matrix(y_test, y_pred_sm)

In [235]:
# Display the confusion matrix
# YOUR CODE HERE
balanced_accuracy_score(y_test, y_pred_sm) 

0.994748035609574

In [236]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred_sm))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   SMOTE oversampling and Random Naive Oversampling 

2. Which model had the best recall score?

    Almost all models have similar scores

3. Which model had the best geometric mean score?

    Similar in all models.
